In [1]:
#hide
#skip
%config Completer.use_jedi = False
%config IPCompleter.greedy=True
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [2]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [3]:
# default_exp fastai.data.load

In [4]:
# export
# Python native modules
import os
from typing import Callable
# Third party libs
from fastcore.all import *
import torchdata.datapipes as dp
from torch.utils.data.dataloader_experimental import DataLoader2
from torch.utils.data.graph import traverse
# Local modules
from fastrl.fastai.data.loop.core import *

from fastrl.fastai.data.pipes.map.mux import *
from fastrl.fastai.data.pipes.map.demux import *

NameError: name 'Loop' is not defined

# Loading
> Objects using the `Loop` and `DataPipe` API for DataLoading

We will replicate [fastai mnist loading](https://colab.research.google.com/github/fastai/fastbook/blob/master/04_mnist_basics.ipynb).

In [ ]:
from fastai.vision.all import untar_data,URLs,get_image_files

path = untar_data(URLs.MNIST_SAMPLE)

(path/'train').ls()

First we create the dataset...

In [ ]:
# export
class TypeTransformLoop(MapLoop):
    def __init__(self,datapipe, type_tfms):
        self.type_tfms,self.datapipe = Pipeline(type_tfms),datapipe
    
    def __subiter__(self):
        for data in self.datapipe:
            yield self.type_tfms(data)
            
    def __len__(self): return len(self.datapipe)
    
class ItemTransformLoop(IterLoop):
    def __init__(self,source_datapipe, item_tfms:List[Callable]): 
        self.item_tfms,self.source_datapipe = Pipeline(item_tfms),source_datapipe
    
    def __subiter__(self):
        for data in self.source_datapipe:
            yield self.item_tfms(data)
    
    
class BatchTransformLoop(IterLoop):
    def __init__(self,source_datapipe, batch_tfms):
        self.batch_tfms,self.source_datapipe = Pipeline(batch_tfms),source_datapipe
    
    def __subiter__(self):
        for data in self.source_datapipe:
            yield self.batch_tfms(data)

In [ ]:
# export
def default_loader_loop(
    items,
    splitter,
    type_tfms=None,
    item_tfms=None,
    batch_tfms=None,
    bs=2
):
    base = dp.map.SequenceWrapper(items)
    type_tfm_dp = TypeTransformLoop(base, ifnone(type_tfms,L()))
    trainer,validater = DemultiplexerMapDataPipe(
        type_tfm_dp,
        num_instances=2,
        classifier_fn=splitter,
        drop_none=True
    )
    train_iter,valid_iter = L(trainer,validater).map(dp.iter.MapToIterConverter)
    train_item_tfm_dp,valid_item_tfm_dp = L(trainer,validater).map(ItemTransformLoop, item_tfms=ifnone(item_tfms,L()))
    train_b,valid_b = trainer.cb_batch(bs),validater.cb_batch(bs)
    return train_b,valid_b

In [ ]:
def GrandparentSplitter(train='train',valid='valid'):
    def splitter(item):
        if all(s not in item.parts for s in (train,valid)): return None
        if item.is_dir(): return None
        # valid=1, train=0
        return valid in item.parts 
    return splitter

In [ ]:
base = default_loader_loop(
    L(path.rglob('*')),
    GrandparentSplitter()
)

In [ ]:
base[0][6]

In [ ]:
list(base[0])[:5],list(base[1])[:5]

In [ ]:
len(type_tfm_dp)

In [ ]:
len(train)

In [ ]:
train[0]

In [ ]:
class 

In [ ]:
datapipe = 

In [ ]:
train_dl,valid_dl = DataLoader2(),DataLoader2()

In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbdev.cli import *
    make_readme()
    notebook2script(silent=True)